# ---------------------------------------
# Step 1: Reading and Combining Data
# ---------------------------------------


In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [10]:
# Load all CSV files into a list of DataFrames
try:
    csv_files = glob.glob(data_path)
    if len(csv_files) == 0:
        print("Error: No CSV files found in data\\PassengerVehicle_Stats")
        raise FileNotFoundError("No CSV files found.")
    if len(csv_files) != 9:
        print(f"Warning: Expected 9 CSV files, found {len(csv_files)}")
    df_list = [pd.read_csv(file) for file in csv_files]
    print(f"Loaded {len(df_list)} CSV files:")
    for file in csv_files:
        print(f"- {os.path.basename(file)}")
except Exception as e:
    print(f"Error loading CSV files: {e}")
    raise

# Concatenate into a single DataFrame
vehicles_df = pd.concat(df_list, ignore_index=True)
print("\nCombined DataFrame shape:", vehicles_df.shape)
print("Columns:", vehicles_df.columns.tolist())

Loaded 9 CSV files:
- Bio-Diesel_PassengerVehicle_Stats.csv
- Compressed Natural Gas_PassengerVehicle_Stats.csv
- Diesel_PassengerVehicle_Stats.csv
- Electric_PassengerVehicle_Stats.csv
- Flex Fuel_PassengerVehicle_Stats.csv
- Gasoline_PassengerVehicle_Stats.csv
- Horse_PassengerVehicle_Stats.csv
- Hybrid_PassengerVehicle_Stats.csv
- Pedal_PassengerVehicle_Stats.csv

Combined DataFrame shape: (16602, 17)
Columns: ['Unnamed: 0', 'Public Vehicle Number', 'Status', 'Vehicle Make', 'Vehicle Model', 'Vehicle Model Year', 'Vehicle Color', 'Vehicle Fuel Source', 'Wheelchair Accessible', 'Company Name', 'Address', 'City', 'State', 'ZIP Code', 'Taxi Affiliation', 'Taxi Medallion License Management ', 'Record ID']


# ---------------------------------------
# Step 2: Initial Data Exploration and Cleaning
# ---------------------------------------

In [31]:
# Drop unnecessary index column
# Reason: 'Unnamed: 0' is likely an index from the CSV files, redundant for analysis.
if 'Unnamed: 0' in vehicles_df.columns:
    vehicles_df = vehicles_df.drop(columns=['Unnamed: 0'])
    print("Dropped 'Unnamed: 0' column.")



In [32]:
# Rename column with trailing space
# Reason: Standardizes column name for consistency and ease of use.
vehicles_df = vehicles_df.rename(columns={'Taxi Medallion License Management ': 'Taxi Medallion License Management'})


In [33]:
# Examine DataFrame structure
print("\nDataFrame Info:")
print(vehicles_df.info())


DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 16576 entries, 0 to 16601
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Status                             16576 non-null  object 
 1   Vehicle Make                       16576 non-null  object 
 2   Vehicle Model                      16576 non-null  object 
 3   Vehicle Model Year                 16576 non-null  float64
 4   Vehicle Color                      16576 non-null  object 
 5   Vehicle Fuel Source                16576 non-null  object 
 6   Wheelchair Accessible              16576 non-null  object 
 7   Company Name                       16576 non-null  object 
 8   City                               16576 non-null  object 
 9   State                              14790 non-null  object 
 10  ZIP Code                           16576 non-null  object 
 11  Taxi Affiliation                   16576 n

In [34]:
# Check initial null values
print("\nInitial Null Values:")
print(vehicles_df.isnull().sum())


Initial Null Values:
Status                                  0
Vehicle Make                            0
Vehicle Model                           0
Vehicle Model Year                      0
Vehicle Color                           0
Vehicle Fuel Source                     0
Wheelchair Accessible                   0
Company Name                            0
City                                    0
State                                1786
ZIP Code                                0
Taxi Affiliation                        0
Taxi Medallion License Management       0
Record ID                               0
Vehicle Type                            0
dtype: int64


In [35]:
# Remove duplicate records
# Reason: Duplicates may result from data entry errors or merging issues, leading to
# overcounting of vehicles. Removing them ensures each vehicle is represented once.
initial_rows = vehicles_df.shape[0]
vehicles_df = vehicles_df.drop_duplicates()
print(f"\nRemoved {initial_rows - vehicles_df.shape[0]} duplicate rows. New shape:", vehicles_df.shape)



Removed 932 duplicate rows. New shape: (15644, 15)


In [37]:
# Evaluate null records
# Reason: Nulls are removed only if critical columns (`Public Vehicle Number`, `Record ID`)
# are missing, as these are unique identifiers. Other nulls are retained for imputation.
print(f"\nNumber of rows with any null values: {len(vehicles_df[vehicles_df.isnull().any(axis=1)])}")
print("\nColumns before null check:", vehicles_df.columns.tolist())
critical_columns = [col for col in ['Public Vehicle Number', 'Record ID'] if col in vehicles_df.columns]
if not critical_columns:
    print("No critical columns ('Public Vehicle Number', 'Record ID') found. Skipping null check.")
else:
    null_critical = vehicles_df[critical_columns].isnull().any(axis=1)
    if null_critical.sum() > 0:
        print(f"Found {null_critical.sum()} rows with nulls in critical columns: {critical_columns}")
        vehicles_df = vehicles_df[~null_critical]
        print("Removed rows with nulls in critical columns. New shape:", vehicles_df.shape)
    else:
        print("No rows with nulls in critical columns. Retaining all rows for imputation.")


Number of rows with any null values: 1694

Columns before null check: ['Status', 'Vehicle Make', 'Vehicle Model', 'Vehicle Model Year', 'Vehicle Color', 'Vehicle Fuel Source', 'Wheelchair Accessible', 'Company Name', 'City', 'State', 'ZIP Code', 'Taxi Affiliation', 'Taxi Medallion License Management', 'Record ID', 'Vehicle Type']
No rows with nulls in critical columns. Retaining all rows for imputation.



# ---------------------------------------
# Step 3: Handle Outliers and Missing Values
# ---------------------------------------

In [39]:
# Outlier detection for Vehicle Model Year
def detect_outliers(df, column):
    """
    Detect outliers using the IQR method.
    Returns indices and values of outliers for inspection.
    """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)][column]
    return outliers

# Check outliers for Vehicle Model Year
print("\nOutlier Detection for Vehicle Model Year:")
if 'Vehicle Model Year' in vehicles_df.columns:
    # Ensure Vehicle Model Year is numeric
    vehicles_df['Vehicle Model Year'] = pd.to_numeric(vehicles_df['Vehicle Model Year'], errors='coerce')
    outliers = detect_outliers(vehicles_df, 'Vehicle Model Year')
    print(f"Outliers in Vehicle Model Year:\n{outliers}\n")
    
    # Handle invalid years
    # Reason: Years <1900 or >2026 are invalid. Retain older but plausible years (e.g., 1980s).
    current_year = 2025
    invalid_years = vehicles_df[
        (vehicles_df['Vehicle Model Year'] < 1900) | 
        (vehicles_df['Vehicle Model Year'] > current_year + 1)
    ]
    if not invalid_years.empty:
        print("Invalid Vehicle Model Years detected:\n", invalid_years[['Vehicle Model Year']])
        vehicles_df = vehicles_df[
            (vehicles_df['Vehicle Model Year'] >= 1900) & 
            (vehicles_df['Vehicle Model Year'] <= current_year + 1) | 
            vehicles_df['Vehicle Model Year'].isnull()
        ]
        print("Removed invalid years. New shape:", vehicles_df.shape)
    else:
        print("No invalid Vehicle Model Years detected.")
else:
    print("Vehicle Model Year column not found. Skipping outlier detection.")

# Impute missing values
# Reason: Imputation preserves data for analysis.
# - Categorical: 'Unknown' avoids bias.
# - Binary: 'No' for conservative imputation.
# - Taxi-related: 'None' for non-taxi vehicles.
# - Numerical: Median for robustness.
columns_to_impute = {
    'Status': 'Unknown',
    'Vehicle Make': 'Unknown',
    'Vehicle Model': 'Unknown',
    'Vehicle Color': 'Unknown',
    'Vehicle Fuel Source': 'Unknown',
    'Wheelchair Accessible': 'No',
    'Company Name': 'Unknown',
    'City': 'Unknown',
    'State': 'Unknown',
    'ZIP Code': 'Unknown',
    'Taxi Affiliation': 'None',
    'Taxi Medallion License Management': 'None'
}
for col, value in columns_to_impute.items():
    if col in vehicles_df.columns:
        vehicles_df[col] = vehicles_df[col].fillna(value)

# Impute Vehicle Model Year
if 'Vehicle Model Year' in vehicles_df.columns and vehicles_df['Vehicle Model Year'].isnull().sum() > 0:
    median_year = vehicles_df['Vehicle Model Year'].median()
    vehicles_df['Vehicle Model Year'] = vehicles_df['Vehicle Model Year'].fillna(median_year)
    print(f"Imputed {vehicles_df['Vehicle Model Year'].isnull().sum()} missing Vehicle Model Years with median: {median_year}")
else:
    print("No missing Vehicle Model Years or column not found.")

# Verify null values after imputation
print("\nNull Values After Imputation:")
print(vehicles_df.isnull().sum())



Outlier Detection for Vehicle Model Year:
Outliers in Vehicle Model Year:
102      2001.0
104      2002.0
106      2000.0
107      1993.0
109      2000.0
          ...  
10400    1995.0
10401    1991.0
12496    2000.0
13320    2000.0
16588    1985.0
Name: Vehicle Model Year, Length: 249, dtype: float64

No invalid Vehicle Model Years detected.
No missing Vehicle Model Years or column not found.

Null Values After Imputation:
Status                               0
Vehicle Make                         0
Vehicle Model                        0
Vehicle Model Year                   0
Vehicle Color                        0
Vehicle Fuel Source                  0
Wheelchair Accessible                0
Company Name                         0
City                                 0
State                                0
ZIP Code                             0
Taxi Affiliation                     0
Taxi Medallion License Management    0
Record ID                            0
Vehicle Type            

# ---------------------------------------
# Step 4: Adding New Column (Vehicle Type)
# ---------------------------------------

In [47]:
# Debug Record ID format
if 'Record ID' in vehicles_df.columns:
    print("\nSample Record ID values:")
    print(vehicles_df['Record ID'].head(10))
    print("\nUnique Record ID patterns:")
    print(vehicles_df['Record ID'].str.split('_', n=1).str[0].value_counts().head(10))
    
    # Inspect all unique Record ID suffixes
    print("\nAll unique Record ID suffixes:")
    suffixes = vehicles_df['Record ID'].str.extract(r'(\D+)$')[0].value_counts()
    print(suffixes)
    
    # Extract Vehicle Type using regex
    # Updated to include all known types
    vehicles_df['Vehicle Type'] = vehicles_df['Record ID'].str.extract(
        r'(Taxi|Pedicab|Bus|Charter\sSightseeing|Livery|Ambulance|Shuttle|Jitney|Medicar|Low\sSpeed\sElectric|Horse\sDrawn\sCarriage)'
    )
    print("\nUnmatched Record ID values (resulting in nan Vehicle Type):")
    print(vehicles_df[vehicles_df['Vehicle Type'].isna()]['Record ID'].head(10))
    
    # Impute nan with 'Unknown'
    vehicles_df['Vehicle Type'] = vehicles_df['Vehicle Type'].fillna('Unknown')
    print("\nUnique Vehicle Types:", vehicles_df['Vehicle Type'].unique())
    print("\nSample of Vehicle Type column:")
    print(vehicles_df[['Record ID', 'Vehicle Type']].head())
else:
    print("Record ID column not found. Skipping Vehicle Type creation.")


Sample Record ID values:
0     12009Charter Sightseeing
1     12248Charter Sightseeing
2     13527Charter Sightseeing
4     13528Charter Sightseeing
5     12025Charter Sightseeing
6     13513Charter Sightseeing
7     13830Charter Sightseeing
8     12026Charter Sightseeing
9                      256Taxi
10                    1616Taxi
Name: Record ID, dtype: object

Unique Record ID patterns:
Record ID
117Pedicab                  1
12009Charter Sightseeing    1
12248Charter Sightseeing    1
13527Charter Sightseeing    1
13528Charter Sightseeing    1
12025Charter Sightseeing    1
13513Charter Sightseeing    1
13830Charter Sightseeing    1
12026Charter Sightseeing    1
256Taxi                     1
Name: count, dtype: int64

All unique Record ID suffixes:
0
Taxi                    6999
Livery                  6001
Charter Sightseeing      976
Ambulance                781
Medicar                  616
Pedicab                  180
Horse Drawn Carriage      56
Jitney                    29
Low

# ---------------------------------------
# Step 5: Column Removal
# ---------------------------------------

In [48]:
# Drop specified columns
columns_to_drop = ['Address', 'Public Vehicle Number']
vehicles_df = vehicles_df.drop(columns=[col for col in columns_to_drop if col in vehicles_df.columns])
print("\nColumns after removal:", vehicles_df.columns.tolist())


Columns after removal: ['Status', 'Vehicle Make', 'Vehicle Model', 'Vehicle Model Year', 'Vehicle Color', 'Vehicle Fuel Source', 'Wheelchair Accessible', 'Company Name', 'City', 'State', 'ZIP Code', 'Taxi Affiliation', 'Taxi Medallion License Management', 'Record ID', 'Vehicle Type']


In [49]:
# Save the processed DataFrame
output_path = 'data\\processed_vehicles_task02.csv'
vehicles_df.to_csv(output_path, index=False)
print(f"\nProcessed DataFrame saved to {output_path}")


Processed DataFrame saved to data\processed_vehicles_task02.csv


In [50]:
# Check final null values
print("\nFinal Null Values:")
print(vehicles_df.isnull().sum())

# Check data ranges for Vehicle Model Year
if 'Vehicle Model Year' in vehicles_df.columns:
    print("\nVehicle Model Year Summary:")
    print(vehicles_df['Vehicle Model Year'].describe())

# Check Vehicle Type distribution
if 'Vehicle Type' in vehicles_df.columns:
    print("\nVehicle Type Distribution:")
    print(vehicles_df['Vehicle Type'].value_counts())



Final Null Values:
Status                               0
Vehicle Make                         0
Vehicle Model                        0
Vehicle Model Year                   0
Vehicle Color                        0
Vehicle Fuel Source                  0
Wheelchair Accessible                0
Company Name                         0
City                                 0
State                                0
ZIP Code                             0
Taxi Affiliation                     0
Taxi Medallion License Management    0
Record ID                            0
Vehicle Type                         0
dtype: int64

Vehicle Model Year Summary:
count    15644.000000
mean      2016.152582
std          5.177405
min       1980.000000
25%       2013.000000
50%       2016.000000
75%       2020.000000
max       2025.000000
Name: Vehicle Model Year, dtype: float64

Vehicle Type Distribution:
Vehicle Type
Taxi                    6999
Livery                  6001
Charter Sightseeing      976
Ambulanc